In [3]:
!pip install imgaug


   ---------------------------------------- 0.0/948.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/948.0 kB ? eta -:--:--
   ---------------------- ----------------- 524.3/948.0 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 948.0/948.0 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------ --------- 1.3/1.7 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 7.7 MB/s eta 0:00:00


In [5]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from imgaug import augmenters as iaa

input_dir = "Resized_IMG_CLASSES"
output_dir = "Augmented_IMG_CLASSES"

# Folder names to augment and class mapping
target_folders = [
    "1. Eczema",              # Class 0
    "3. Atopic Dermatitis",   # Class 2
    "6. Fungal Infections",   # Class 5
    "7. Viral Infections"     # Class 6
]

target_count = 1500

augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Rotate((-25, 25)),
    iaa.Multiply((0.8, 1.2)),
    iaa.GaussianBlur(sigma=(0, 1.0))
])

os.makedirs(output_dir, exist_ok=True)

for folder in os.listdir(input_dir):
    input_path = os.path.join(input_dir, folder)
    output_path = os.path.join(output_dir, folder)
    os.makedirs(output_path, exist_ok=True)

    images = [f for f in os.listdir(input_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    current_count = len(images)

    # Copy originals
    for img in images:
        img_path = os.path.join(input_path, img)
        out_path = os.path.join(output_path, img)
        cv2.imwrite(out_path, cv2.imread(img_path))

    # Skip augmentation if not in the target list
    if folder not in target_folders:
        continue

    print(f"\nAugmenting: {folder}")
    needed = target_count - current_count
    if needed <= 0:
        continue

    idx = 0
    while len(os.listdir(output_path)) < target_count:
        img_name = images[idx % current_count]
        img_path = os.path.join(input_path, img_name)
        image = cv2.imread(img_path)

        if image is None:
            idx += 1
            continue

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        aug_img = augmenter(image=image)
        aug_img = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

        aug_name = f"aug_{idx}_{img_name}"
        aug_path = os.path.join(output_path, aug_name)
        cv2.imwrite(aug_path, aug_img)

        idx += 1



Augmenting: 1. Eczema

Augmenting: 3. Atopic Dermatitis

Augmenting: 6. Fungal Infections

Augmenting: 7. Viral Infections
